In [28]:
import numpy as np
import pandas as pd
import pickle as pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import functions_neuralNet

pd.set_option('display.max_columns', None)

In [29]:
# pull in train + test + features
# train and test are incorrectly labelled but for consitency I will stick with it
df_test = pd.read_csv('../../data/usnw_nb15/test.csv', index_col = 0)
df_train = pd.read_csv('../../data/usnw_nb15/train.csv', index_col = 0)
features = pd.read_csv('../../data/usnw_nb15/features.csv', encoding='cp1252')

In [30]:
# concat train and test together
df_concat = pd.concat([df_train, df_test], axis = 0).reset_index(drop=True)

In [31]:
# take out attack_cat and label so it does not get one hot encoded
df_train_noGT, train_GT = functions_neuralNet.remove_ground_truth(df_train)
df_concat_noGT, concat_GT = functions_neuralNet.remove_ground_truth(df_concat)

In [32]:
# now we have 4 dfs
# first process train and concat dfs in preprocess() and pass in features
df_train_processed = functions_neuralNet.preprocess(df_train_noGT, features)
df_train_nn = df_train_processed
df_concat_preprocessed = functions_neuralNet.preprocess(df_concat_noGT, features)

In [33]:
# next, normalize the preprocessed data
# leaving nn out makes it not normalized ('nn')
df_train_normalized = functions_neuralNet.normalization(df_train_processed, df_train_processed.columns)
df_concat_normalized = functions_neuralNet.normalization(df_concat_preprocessed, df_concat_preprocessed.columns)

In [34]:
# now add the GT back into the dfs so that we can separate the normals from the attacks
# 82k normalized:
df_train_normalized_GT = pd.concat([df_train_normalized, train_GT], axis = 1)

# 82k not normalized
df_train_nn_GT = pd.concat([df_train_nn, train_GT], axis = 1)

# 250k normalized:
df_concat_normalized_GT = pd.concat([df_concat_normalized, concat_GT], axis = 1)

In [35]:
# isolate the normal traffic from the attack traffic
# 82k normalized:
df_train_normal, df_train_attack = functions_neuralNet.separate_traffic_cats(df_train_normalized_GT)

# 82k not normalized:
df_train_normal_nn, df_train_attack_nn = functions_neuralNet.separate_traffic_cats(df_train_nn_GT)

# 250k normalized
df_concat_normal, df_concat_attack = functions_neuralNet.separate_traffic_cats(df_concat_normalized_GT)

In [36]:
# # map HDBSCAN into a new column attack_cat_bc using hdbscan_df_transformer

# 250k normalized 500/10000:
print('df_concat_normal shape: ' + str(df_concat_normal.shape))
df_concat_normal_500_10000 = functions_neuralNet.hdbscan_df_transformer('HDBSCAN_93k_500_10000', df_concat_normal)
df_concat_normal_500_10000.reset_index(inplace=True, drop=True)
# # in the df_train_attack df, duplicate 'attack_cat' into the new column attack_cat_bc
df_concat_attack_500_10000 = df_concat_attack.copy()
df_concat_attack_500_10000['attack_cat_bc_500_10000'] = df_concat_attack_500_10000['attack_cat']
df_concat_attack_500_10000.reset_index(inplace=True, drop=True)
print('df_concat_normal_500_10000 shape: ' + str(df_concat_normal_500_10000.shape))
df_concat_normal_500_10000.head(500)

df_concat_normal shape: (93000, 197)
df_concat_normal_500_10000 shape: (93000, 198)


dur     spkts     dpkts    sbytes    dbytes      rate      sttl  \
0    1.833334e-07  0.000094  0.000000  0.000033  0.000000  0.090909  0.996078   
1    1.333334e-07  0.000094  0.000000  0.000121  0.000000  0.125000  0.996078   
2    8.333335e-08  0.000094  0.000000  0.000073  0.000000  0.200000  0.996078   
3    1.000000e-07  0.000094  0.000000  0.000061  0.000000  0.166667  0.996078   
4    1.666667e-07  0.000094  0.000000  0.000146  0.000000  0.100000  0.996078   
..            ...       ...       ...       ...       ...       ...       ...   
495  1.908424e-02  0.001221  0.001634  0.000116  0.000694  0.000027  0.121569   
496  1.389334e-04  0.001973  0.002178  0.000132  0.000294  0.005398  0.121569   
497  4.213334e-04  0.004227  0.004357  0.000197  0.002140  0.003679  0.121569   
498  7.340001e-05  0.000282  0.000363  0.000038  0.000021  0.001589  0.121569   
499  7.861668e-05  0.000282  0.000363  0.000038  0.000022  0.001484  0.121569   

         dttl     sload     dload     sloss     dloss        sinpkt    dinpkt  \
0    0.000000  0.030121  0.000000  0.000000  0.000000  1.303758e-07  0.000000   
1    0.000000  0.147128  0.000000  0.000000  0.000000  9.481875e-08  0.000000   
2    0.000000  0.142685  0.000000  0.000000  0.000000  5.926172e-08  0.000000   
3    0.000000  0.100200  0.000000  0.000000  0.000000  7.111406e-08  0.000000   
4    0.000000  0.142017  0.000000  0.000000  0.000000  1.185234e-07  0.000000   
..        ...       ...       ...       ...       ...           ...       ...   
495  0.114173  0.000002  0.002992  0.000564  0.000908  1.043655e-03  0.001166   
496  0.114173  0.000294  0.176892  0.001128  0.001090  4.507838e-06  0.000006   
497  0.114173  0.000148  0.433373  0.001316  0.003087  6.565677e-06  0.000009   
498  0.114173  0.000129  0.018471  0.000000  0.000000  1.427813e-05  0.000017   
499  0.114173  0.000121  0.018153  0.000000  0.000000  1.434924e-05  0.000018   

         sjit      djit  swin     stcpb     dtcpb  dwin    tcprtt    synack  \
0    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
1    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
2    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
3    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
4    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
..        ...       ...   ...       ...       ...   ...       ...       ...   
495  0.005951  0.016690   1.0  0.721424  0.221702   1.0  0.000167  0.000150   
496  0.000018  0.000051   1.0  0.210389  0.709995   1.0  0.000213  0.000205   
497  0.000022  0.000068   1.0  0.985509  0.985598   1.0  0.000177  0.000170   
498  0.000001  0.000003   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
499  0.000001  0.000003   0.0  0.000000  0.000000   0.0  0.000000  0.000000   

       ackdat     smean     dmean  trans_depth  response_body_len  ct_srv_src  \
0    0.000000  0.151351  0.000000          0.0                0.0    0.016129   
1    0.000000  0.579054  0.000000          0.0                0.0    0.016129   
2    0.000000  0.344595  0.000000          0.0                0.0    0.032258   
3    0.000000  0.287838  0.000000          0.0                0.0    0.032258   
4    0.000000  0.702027  0.000000          0.0                0.0    0.032258   
..        ...       ...       ...          ...                ...         ...   
495  0.000053  0.064865  0.376667          0.0                0.0    0.080645   
496  0.000051  0.042568  0.120000          0.0                0.0    0.161290   
497  0.000043  0.025676  0.435333          0.0                0.0    0.193548   
498  0.000000  0.079730  0.050667          0.0                0.0    0.145161   
499  0.000000  0.079730  0.053333          0.0                0.0    0.274194   

     ct_state_ttl  ct_dst_ltm  ct_src_dport_ltm  ct_dst_sport_ltm  \
0        0.333333    0.000000          0.000000               0.0   
1       

In [37]:
# # concat the normal traffic and attack traffic back together

# 250k normalized 500/10000:
df_concat_500_10000 = pd.concat([df_concat_normal_500_10000, df_concat_attack_500_10000], axis = 0).reset_index(drop=True)

In [38]:
df_concat_500_10000.head(500)

dur     spkts     dpkts    sbytes    dbytes      rate      sttl  \
0    1.833334e-07  0.000094  0.000000  0.000033  0.000000  0.090909  0.996078   
1    1.333334e-07  0.000094  0.000000  0.000121  0.000000  0.125000  0.996078   
2    8.333335e-08  0.000094  0.000000  0.000073  0.000000  0.200000  0.996078   
3    1.000000e-07  0.000094  0.000000  0.000061  0.000000  0.166667  0.996078   
4    1.666667e-07  0.000094  0.000000  0.000146  0.000000  0.100000  0.996078   
..            ...       ...       ...       ...       ...       ...       ...   
495  1.908424e-02  0.001221  0.001634  0.000116  0.000694  0.000027  0.121569   
496  1.389334e-04  0.001973  0.002178  0.000132  0.000294  0.005398  0.121569   
497  4.213334e-04  0.004227  0.004357  0.000197  0.002140  0.003679  0.121569   
498  7.340001e-05  0.000282  0.000363  0.000038  0.000021  0.001589  0.121569   
499  7.861668e-05  0.000282  0.000363  0.000038  0.000022  0.001484  0.121569   

         dttl     sload     dload     sloss     dloss        sinpkt    dinpkt  \
0    0.000000  0.030121  0.000000  0.000000  0.000000  1.303758e-07  0.000000   
1    0.000000  0.147128  0.000000  0.000000  0.000000  9.481875e-08  0.000000   
2    0.000000  0.142685  0.000000  0.000000  0.000000  5.926172e-08  0.000000   
3    0.000000  0.100200  0.000000  0.000000  0.000000  7.111406e-08  0.000000   
4    0.000000  0.142017  0.000000  0.000000  0.000000  1.185234e-07  0.000000   
..        ...       ...       ...       ...       ...           ...       ...   
495  0.114173  0.000002  0.002992  0.000564  0.000908  1.043655e-03  0.001166   
496  0.114173  0.000294  0.176892  0.001128  0.001090  4.507838e-06  0.000006   
497  0.114173  0.000148  0.433373  0.001316  0.003087  6.565677e-06  0.000009   
498  0.114173  0.000129  0.018471  0.000000  0.000000  1.427813e-05  0.000017   
499  0.114173  0.000121  0.018153  0.000000  0.000000  1.434924e-05  0.000018   

         sjit      djit  swin     stcpb     dtcpb  dwin    tcprtt    synack  \
0    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
1    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
2    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
3    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
4    0.000000  0.000000   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
..        ...       ...   ...       ...       ...   ...       ...       ...   
495  0.005951  0.016690   1.0  0.721424  0.221702   1.0  0.000167  0.000150   
496  0.000018  0.000051   1.0  0.210389  0.709995   1.0  0.000213  0.000205   
497  0.000022  0.000068   1.0  0.985509  0.985598   1.0  0.000177  0.000170   
498  0.000001  0.000003   0.0  0.000000  0.000000   0.0  0.000000  0.000000   
499  0.000001  0.000003   0.0  0.000000  0.000000   0.0  0.000000  0.000000   

       ackdat     smean     dmean  trans_depth  response_body_len  ct_srv_src  \
0    0.000000  0.151351  0.000000          0.0                0.0    0.016129   
1    0.000000  0.579054  0.000000          0.0                0.0    0.016129   
2    0.000000  0.344595  0.000000          0.0                0.0    0.032258   
3    0.000000  0.287838  0.000000          0.0                0.0    0.032258   
4    0.000000  0.702027  0.000000          0.0                0.0    0.032258   
..        ...       ...       ...          ...                ...         ...   
495  0.000053  0.064865  0.376667          0.0                0.0    0.080645   
496  0.000051  0.042568  0.120000          0.0                0.0    0.161290   
497  0.000043  0.025676  0.435333          0.0                0.0    0.193548   
498  0.000000  0.079730  0.050667          0.0                0.0    0.145161   
499  0.000000  0.079730  0.053333          0.0                0.0    0.274194   

     ct_state_ttl  ct_dst_ltm  ct_src_dport_ltm  ct_dst_sport_ltm  \
0        0.333333    0.000000          0.000000               0.0   
1       

In [39]:
# split the data using train_test_splitter

# 250 normalized 500/10000:
df_concat_500_10000_split = functions_neuralNet.train_test_splitter(df_concat_500_10000)

In [40]:
# dump them to csv

df_concat_500_10000_split.to_csv('250k_normalized_cats_train_test_split_500_10000.csv')